# Lab 1 - Getting Started

## Project Overview

In this project, you will use both `polars` and `pyspark` to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  While the MinneMUDAC 2016 site, is no longer live, a copy was obtained using the [Wayback Machine (https://web.archive.org) and has been provided in [the overview notebook](./MinneMUDAC_2016_Overview.ipynb).  You should document your work in a Jupyter notebook, which will be used to submit your solution.

## Lab 1 Tasks

In this lab, you will perform the following tasks

1. Download and unzip the data.
2. Investigating the columns in various property data files.

### Task 1 - Data download and unzip

While the download links on the original site no longer work, you can access the data by clicking on [this link](https://mnscu-my.sharepoint.com/:u:/g/personal/bn8210wy_minnstate_edu/EdUePet8JsdKv5aUt9gvjoMBxQhXrOx73WpQyVNwLVDfkA?e=rR8qrc)

1. Move the zip file unto your repository
2. Unzip and move the files into your data folder.

**Hint.** Take a look the the Colab section of any module 5 lecture for an example.

In [58]:
import polars as pl
import polars.selectors as cs
from glob import glob
import re
from functools import reduce
from operator import mul, add

In [4]:
# Perform the tasks listed above then use `glob` to verify the contents/paths.
sorted(glob('./data/**/*.txt', recursive = True))

['./data\\MinneMUDAC_raw_files\\2002_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2003_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2004_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2005_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2006_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2007_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2008_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2009_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2010_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2011_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2012_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2013_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2014_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\2015_metro_tax_parcels.txt',
 './data\\MinneMUDAC_raw_files\\Parcel_Lake_Monitoring_Site_Xref.txt',
 './data\\MinneMUDAC_raw_files\\mces_lakes_1999_2014.txt']

In [21]:
(parcel_paths := [p.replace('\\','/') for p in sorted(glob('./data/**/*.txt', recursive = True)) if 'parcels' in p])

['./data/MinneMUDAC_raw_files/2002_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2003_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2012_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2014_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2015_metro_tax_parcels.txt']

#### Questions

1. Notice that we have multiple property files, one per year.  What verb(s) will be used to combine these files?
2. Why is it important to compare the columns of these files?
3. Use the `%%bash` cell magic along with `head path` to inspect the first few lines of one of the files, where `...` is replaced with the specific path to one of the files.  How are the columns separated?

> <font color="orange"> 

1. We would use union to combine all of these files together. We would use na combination of select, filter, and mutate to make them comptable first.

2. we have to investigate these columns so that we can correct any columns that do not match before joining the tables together.

3. Each column is seperated with a |.

<font color="orange">

In [17]:
%%bash

head ./data/MinneMUDAC_raw_files/2015_metro_tax_parcels.txt

wsl: Failed to start the systemd user session for 'cv0437li'. See journalctl for more details.


ACRES_DEED|ACRES_POLY|AGPRE_ENRD|AGPRE_EXPD|AG_PRESERV|BASEMENT|BLDG_NUM|BLOCK|CITY|CITY_USPS|COOLING|COUNTY_ID|DWELL_TYPE|EMV_BLDG|EMV_LAND|EMV_TOTAL|FIN_SQ_FT|GARAGE|GARAGESQFT|GREEN_ACRE|HEATING|HOMESTEAD|HOME_STYLE|LANDMARK|LOT|MULTI_USES|NUM_UNITS|OPEN_SPACE|OWNER_MORE|OWNER_NAME|OWN_ADD_L1|OWN_ADD_L2|OWN_ADD_L3|PARC_CODE|PIN|PLAT_NAME|PREFIXTYPE|PREFIX_DIR|SALE_DATE|SALE_VALUE|SCHOOL_DST|SPEC_ASSES|STREETNAME|STREETTYPE|SUFFIX_DIR|Shape_Area|Shape_Leng|TAX_ADD_L1|TAX_ADD_L2|TAX_ADD_L3|TAX_CAPAC|TAX_EXEMPT|TAX_NAME|TOTAL_TAX|UNIT_INFO|USE1_DESC|USE2_DESC|USE3_DESC|USE4_DESC|WSHD_DIST|XUSE1_DESC|XUSE2_DESC|XUSE3_DESC|XUSE4_DESC|YEAR_BUILT|Year|ZIP|ZIP4|centroid_long|centroid_lat
30.0|26.7|||N||||SAINT FRANCIS|ELK RIVER||003|AGRICULTURAL|0.0|132100.0|132100.0|0.0|N||||N|||||0|||JONES TRUSTEE RAYMOND|23725 NACRE ST NW|ELK RIVER|MN,  55330|0.0|003-333425210001|||||0.0|15|0.0||||||23725 NACRE ST NW|ELK RIVER|MN,  55330|1080.0|N|JONES TRUSTEE RAYMOND|1410.0||AGRICULTURAL||||UPPER RUM RI

## Task 2 - Perform a column exploration on the property files

<img src="./img/column_master_file.png" width="800">

**Hints.**

1. Use `glob` to get a list of all the parcel files.
2. Use a list comprehension contains pairs of value of the form `(year, df)` where `df` a `polars` or `pyspark` data frame for each file. Keep in mind that the files are large so we will need to leverage the lazy natures of `polars` and `pyspark`. You will want to use `scan_csv` in `polars`.  On the other hand, `pyspark` is lazy and will do minimal work on this step.
3. Perform a column exploration by creating an indicator summary table, with on row per column and one column per year/file.  The values in the table should be `1` if the column is present in that year's file and `0` otherwise.  You may need to pivot or reshape your data frame to get it into this format.
4. There is one problematic year.  Which year is it and what is the problem?  We will be skipping this year in future analysis.
5. Sort the summary table by the column names and inspect the results.  In particular, look for similar names with different spellings or capitalizations.
6. Identify the common columns that are present in all years.

In [28]:
# Glob/path processing code here
year_pattern = re.compile(r'(\d{4})_metro_tax_parcels')
get_year = lambda path: (m.group(1) if (m := year_pattern.search(path)) else None)
[get_year(path) for path in parcel_paths]

['2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015']

In [ ]:
# Your lazy polars/pyspark code here
(
    schemas := {
        path: {col: pl.String for col in pl.scan_csv(path, has_header= True, ignore_errors=True, separator='|').limit(1).collect().columns}
        for path in parcel_paths
    }
)

[
    indicator_tables := [(
        pl.scan_csv(path, has_header= True, separator='|', schema = schemas[path]).limit(1)
        .unpivot()
        .rename({'variable':'column'})
        .drop('value')
        .with_columns(pl.lit(1).alias(get_year(path)))
        .collect()
    ) for path in parcel_paths]
]

[[shape: (75, 2)
  ┌───────────────┬──────┐
  │ column        ┆ 2002 │
  │ ---           ┆ ---  │
  │ str           ┆ i32  │
  ╞═══════════════╪══════╡
  │ ACRES_DEED    ┆ 1    │
  │ ACRES_POLY    ┆ 1    │
  │ AGPRE_ENRD    ┆ 1    │
  │ AGPRE_EXPD    ┆ 1    │
  │ AG_PRESERV    ┆ 1    │
  │ …             ┆ …    │
  │ Year          ┆ 1    │
  │ ZIP           ┆ 1    │
  │ ZIP4          ┆ 1    │
  │ centroid_lat  ┆ 1    │
  │ centroid_long ┆ 1    │
  └───────────────┴──────┘,
  shape: (34, 2)
  ┌───────────────┬──────┐
  │ column        ┆ 2003 │
  │ ---           ┆ ---  │
  │ str           ┆ i32  │
  ╞═══════════════╪══════╡
  │ BLDG_NUM      ┆ 1    │
  │ CITY          ┆ 1    │
  │ COUNTY_ID     ┆ 1    │
  │ EMV_BLDG      ┆ 1    │
  │ EMV_LAND      ┆ 1    │
  │ …             ┆ …    │
  │ YEAR_BUILT    ┆ 1    │
  │ Year          ┆ 1    │
  │ ZIP           ┆ 1    │
  │ centroid_long ┆ 1    │
  │ centroid_lat  ┆ 1    │
  └───────────────┴──────┘,
  shape: (71, 2)
  ┌───────────────┬──────┐
  

In [52]:
# Further exploration (add cells as needed.
join_next = lambda left, right: left.join(right, on = 'column', how = 'full').drop('column_right')

(indicator_table := 
    (reduce(join_next, indicator_tables))
    .filter(pl.col('column').is_not_null())
    .fill_null(0)
)

column,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""ACRES_DEED""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""ACRES_POLY""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""AGPRE_ENRD""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""AGPRE_EXPD""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""AG_PRESERV""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Year""",1,1,1,1,1,1,1,1,1,1,1,1,1,1
"""ZIP""",1,1,1,1,1,1,1,1,1,1,1,1,1,1
"""ZIP4""",1,0,1,1,1,1,1,1,1,1,1,1,1,1


In [54]:
(
    indicator_table
    .select(cs.integer().sum())
)

2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
75,34,70,70,70,70,70,70,70,70,70,70,70,70


In [75]:
pl.Config.set_tbl_rows(200)
pl.Config.set_tbl_cols(25)

polars.config.Config

In [76]:
(
    indicator_table
    .drop('2002', '2003')
    .with_columns(in_all = pl.reduce(mul, cs.integer()), 
                  count = pl.reduce(add, cs.integer())
                  )
    .filter(pl.col('in_all') == 1)
    .get_column('column')
    .to_list()
)

['ACRES_DEED',
 'ACRES_POLY',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'AG_PRESERV',
 'BASEMENT',
 'BLDG_NUM',
 'BLOCK',
 'CITY',
 'CITY_USPS',
 'COOLING',
 'COUNTY_ID',
 'DWELL_TYPE',
 'EMV_BLDG',
 'EMV_LAND',
 'EMV_TOTAL',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'GREEN_ACRE',
 'HEATING',
 'HOMESTEAD',
 'HOME_STYLE',
 'LANDMARK',
 'LOT',
 'MULTI_USES',
 'NUM_UNITS',
 'OPEN_SPACE',
 'OWNER_MORE',
 'OWNER_NAME',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'PARC_CODE',
 'PIN',
 'PLAT_NAME',
 'PREFIXTYPE',
 'PREFIX_DIR',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'Shape_Area',
 'Shape_Leng',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'TAX_CAPAC',
 'TAX_EXEMPT',
 'TAX_NAME',
 'TOTAL_TAX',
 'UNIT_INFO',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'WSHD_DIST',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'YEAR_BUILT',
 'Year',
 'ZIP',
 'ZIP4',
 'centroid_lat',
 'centroid_long']

In [79]:
from column_data import cols_to_keep
cols_to_keep

['ACRES_DEED',
 'ACRES_POLY',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'AG_PRESERV',
 'BASEMENT',
 'BLDG_NUM',
 'BLOCK',
 'CITY',
 'CITY_USPS',
 'COOLING',
 'COUNTY_ID',
 'DWELL_TYPE',
 'EMV_BLDG',
 'EMV_LAND',
 'EMV_TOTAL',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'GREEN_ACRE',
 'HEATING',
 'HOMESTEAD',
 'HOME_STYLE',
 'LANDMARK',
 'LOT',
 'MULTI_USES',
 'NUM_UNITS',
 'OPEN_SPACE',
 'OWNER_MORE',
 'OWNER_NAME',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'PARC_CODE',
 'PIN',
 'PLAT_NAME',
 'PREFIXTYPE',
 'PREFIX_DIR',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'Shape_Area',
 'Shape_Leng',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'TAX_CAPAC',
 'TAX_EXEMPT',
 'TAX_NAME',
 'TOTAL_TAX',
 'UNIT_INFO',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'WSHD_DIST',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'YEAR_BUILT',
 'Year',
 'ZIP',
 'ZIP4',
 'centroid_lat',
 'centroid_long']

In [83]:
column_schema = {col : pl.String for col in cols_to_keep}

#### Your conclusions

<font color="orange">
The 2003 dataset is missing a lot of columns. We will start our analysis in 2004, so that we do not lose out on the information that is missing from 2003. There were 70 columns that were in every single data file. Each of these columns ahd the name spelled the same way. The 5 columns that were not in all of the files were not present in any of the files post 2003. Thus we do not need to worry about these as we aren't including anything before 2004. All of the column names have been stored properly.
</font>